

# Saving/Resuming Study with RDB Backend

An RDB backend enables persistent experiments (i.e., to save and resume a study) as well as access to history of studies.
In addition, we can run multi-node optimization tasks with this feature, which is described in `distributed`.

In this section, let's try simple examples running on a local environment with SQLite DB.

<div class="alert alert-info"><h4>Note</h4><p>You can also utilize other RDB backends, e.g., PostgreSQL or MySQL, by setting the storage argument to the DB's URL.
    Please refer to `SQLAlchemy's document <https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls>`_ for how to set up the URL.</p></div>


## New Study

We can create a persistent study by calling :func:`~optuna.study.create_study` function as follows.
An SQLite file ``example.db`` is automatically initialized with a new study record.


In [1]:
import logging
import sys

import optuna

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "example-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name)

[I 2021-11-30 06:41:10,077] A new study created in RDB with name: example-study


A new study created in RDB with name: example-study


To run a study, call :func:`~optuna.study.Study.optimize` method passing an objective function.



In [2]:
def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    return (x - 2) ** 2


study.optimize(objective, n_trials=3)

[I 2021-11-30 06:41:15,238] Trial 0 finished with value: 31.586258803176968 and parameters: {'x': -3.620165371515056}. Best is trial 0 with value: 31.586258803176968.


Trial 0 finished with value: 31.586258803176968 and parameters: {'x': -3.620165371515056}. Best is trial 0 with value: 31.586258803176968.


[I 2021-11-30 06:41:15,280] Trial 1 finished with value: 40.62144109639532 and parameters: {'x': -4.373495202508222}. Best is trial 0 with value: 31.586258803176968.


Trial 1 finished with value: 40.62144109639532 and parameters: {'x': -4.373495202508222}. Best is trial 0 with value: 31.586258803176968.


[I 2021-11-30 06:41:15,325] Trial 2 finished with value: 34.38119029932718 and parameters: {'x': 7.863547586515111}. Best is trial 0 with value: 31.586258803176968.


Trial 2 finished with value: 34.38119029932718 and parameters: {'x': 7.863547586515111}. Best is trial 0 with value: 31.586258803176968.


## Resume Study

To resume a study, instantiate a :class:`~optuna.study.Study` object
passing the study name ``example-study`` and the DB URL ``sqlite:///example-study.db``.



In [3]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective, n_trials=3)

[I 2021-11-30 06:42:55,061] Using an existing study with name 'example-study' instead of creating a new one.


Using an existing study with name 'example-study' instead of creating a new one.


[I 2021-11-30 06:42:55,134] Trial 3 finished with value: 141.7921150299389 and parameters: {'x': -9.907649433449865}. Best is trial 0 with value: 31.586258803176968.


Trial 3 finished with value: 141.7921150299389 and parameters: {'x': -9.907649433449865}. Best is trial 0 with value: 31.586258803176968.


[I 2021-11-30 06:42:55,174] Trial 4 finished with value: 44.07871875191102 and parameters: {'x': -4.63918057834783}. Best is trial 0 with value: 31.586258803176968.


Trial 4 finished with value: 44.07871875191102 and parameters: {'x': -4.63918057834783}. Best is trial 0 with value: 31.586258803176968.


[I 2021-11-30 06:42:55,212] Trial 5 finished with value: 18.094702837704386 and parameters: {'x': -2.2537868820269296}. Best is trial 5 with value: 18.094702837704386.


Trial 5 finished with value: 18.094702837704386 and parameters: {'x': -2.2537868820269296}. Best is trial 5 with value: 18.094702837704386.


## Experimental History

We can access histories of studies and trials via the :class:`~optuna.study.Study` class.
For example, we can get all trials of ``example-study`` as:



In [4]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-11-30 06:43:46,465] Using an existing study with name 'example-study' instead of creating a new one.


Using an existing study with name 'example-study' instead of creating a new one.


The method :func:`~optuna.study.Study.trials_dataframe` returns a pandas dataframe like:



In [5]:
print(df)

   number       value  params_x     state
0       0   31.586259 -3.620165  COMPLETE
1       1   40.621441 -4.373495  COMPLETE
2       2   34.381190  7.863548  COMPLETE
3       3  141.792115 -9.907649  COMPLETE
4       4   44.078719 -4.639181  COMPLETE
5       5   18.094703 -2.253787  COMPLETE


A :class:`~optuna.study.Study` object also provides properties
such as :attr:`~optuna.study.Study.trials`, :attr:`~optuna.study.Study.best_value`,
:attr:`~optuna.study.Study.best_params` (see also `first`).



In [6]:
print("Best params: ", study.best_params)
print("Best value: ", study.best_value)
print("Best Trial: ", study.best_trial)
print("Trials: ", study.trials)

Best params:  {'x': -2.2537868820269296}
Best value:  18.094702837704386
Best Trial:  FrozenTrial(number=5, values=[18.094702837704386], datetime_start=datetime.datetime(2021, 11, 30, 6, 42, 55, 179067), datetime_complete=datetime.datetime(2021, 11, 30, 6, 42, 55, 192590), params={'x': -2.2537868820269296}, distributions={'x': UniformDistribution(high=10.0, low=-10.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=6, state=TrialState.COMPLETE, value=None)
Trials:  [FrozenTrial(number=0, values=[31.586258803176968], datetime_start=datetime.datetime(2021, 11, 30, 6, 41, 15, 185861), datetime_complete=datetime.datetime(2021, 11, 30, 6, 41, 15, 204601), params={'x': -3.620165371515056}, distributions={'x': UniformDistribution(high=10.0, low=-10.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None), FrozenTrial(number=1, values=[40.62144109639532], datetime_start=datetime.datetime(2021, 11, 30, 6, 41, 15, 243461),